In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import json
from rca import process_categorical, best_logistic_solver, k_fold_cross_val, make_binary_scorer, make_multiclass_scorer, checker

## Loading Data

In [15]:
rca = pd.read_csv('../../data/final/rca.csv')
rca

,embed,embed_type,norm,train_n,p,r2_mean,r2_sd,check
0,CBOW_GoogleNews,text,Freq_HAL,51174,300,0.422344,0.006255,pass
1,CBOW_GoogleNews,text,Freq_KF,26605,300,0.463358,0.009852,pass
2,CBOW_GoogleNews,text,Freq_SUBTLEXUS,43939,300,0.488748,0.006706,pass
3,CBOW_GoogleNews,text,Freq_SUBTLEXUK,47398,300,0.479608,0.008173,pass
4,CBOW_GoogleNews,text,Freq_Blog,53251,300,0.463084,0.006165,pass
...,...,...,...,...,...,...,...,...
7295,THINGS,behavior,familiarity_vanarsdall,376,49,0.060692,0.083942,pass
7296,THINGS,behavior,imageability_vanarsdall,376,49,0.053593,0.095680,pass
7297,THINGS,behavior,familiarity_fear,173,49,0.139160,0.160894,pass
7298,THINGS,behavior,aoa_fear,173,49,-0.021206,0.121789,pass


In [16]:
embed_means = rca.groupby('embed').mean(numeric_only=True)

# Adding embed types
with open('../../data/raw/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_means['type'] = embed_means.index.map(embed_to_type)

# Finding top 2 text 
text_name_1, text_name_2 = (
    embed_means.query('type == "text"').sort_values('r2_mean', ascending=False).head(2).index.tolist()
)
text_name_1, text_name_2

('CBOW_GoogleNews', 'morphoNLM')

In [17]:
# Finding top behavior
behavior_name = (
    embed_means.query('type == "behavior"').sort_values('r2_mean', ascending=False).head(1).index[0]
)
behavior_name

'PPMI_SVD_SWOW'

In [18]:
# Loading embeds
text_1 = pd.read_csv(f'../../data/raw/embeds/{text_name_1}.csv', index_col=0)
text_2 = pd.read_csv(f'../../data/raw/embeds/{text_name_2}.csv', index_col=0)
behavior = pd.read_csv(f'../../data/raw/embeds/{behavior_name}.csv', index_col=0)

# Aligning vocabs
intersect = sorted(list(set.intersection(set(text_1.index), set(text_2.index), set(behavior.index))))
text_1, text_2, behavior = text_1.loc[intersect], text_2.loc[intersect], behavior.loc[intersect]

# Standardizing
standardize = lambda df: (df - df.mean()) / df.std()
text_1, text_2, behavior = standardize(text_1), standardize(text_2), standardize(behavior)

# Ensembling for comparison
embeds = {
    text_name_1: text_1, 
    text_name_2: text_2,
    text_name_1 + '&' + text_name_2: pd.concat([text_1, text_2], axis=1),
    text_name_1 + '&' + behavior_name: pd.concat([text_1, behavior], axis=1),
    text_name_2 + '&' + behavior_name: pd.concat([text_2, behavior], axis=1)
}

# Fixing column names
for embed_name, embed in embeds.items():
    embed.columns = list(range(embed.shape[1]))
    embeds[embed_name] = embed

{name: embed.shape for name, embed in embeds.items()}

{'CBOW_GoogleNews': (11182, 300),
 'morphoNLM': (11182, 50),
 'CBOW_GoogleNews&morphoNLM': (11182, 350),
 'CBOW_GoogleNews&PPMI_SVD_SWOW': (11182, 600),
 'morphoNLM&PPMI_SVD_SWOW': (11182, 350)}

In [19]:
meta = pd.read_csv('../../data/raw/psychNorms_metadata.csv', index_col=0)
meta['associated_embed'] = meta['associated_embed'].str.split(' ')

norms = pd.read_csv('../../data/raw/psychNorms.zip', index_col=0, compression='zip', low_memory=False)
norms

,Freq_HAL,Freq_KF,Freq_SUBTLEXUS,Freq_SUBTLEXUK,Freq_Blog,Freq_Twitter,Freq_News,Freq_CobW,Freq_CobS,CD_SUBTLEXUS,...,reproduction_vanarsdall,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall,familiarity_fear,aoa_fear,imageability_fear
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.62,4.38,2.93
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.79,3.07,5.29
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.29,3.38,5.64
March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.43,2.76,3.50


## Cross Validation

In [20]:
# Ridge
min_alpha, max_alpha = -3, 6 
alphas = np.logspace(min_alpha, max_alpha,  max_alpha - min_alpha + 1)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scorer = make_binary_scorer()
multiclass_scorer = make_multiclass_scorer()

# outer_cv setting 
outer_cv, n_jobs = 5, 6

solo_embed_names = [text_name_1, text_name_2, behavior_name] # For checking data leakage in checker

In [ ]:
# RCA
rca = []
for norm_name in tqdm(norms.columns):
    print(f'{norm_name}:')
    y = norms[norm_name].dropna()
    
    to_print = []
    for embed_name, embed in embeds.items():
        
        # Aligning embed with norm
        X, y = embed.align(y, axis='index', join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators ir categorical or continuous
        if norm_dtype in ['binary', 'multiclass']: # categorical
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(y, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv), solver=solver
            )
            scoring = binary_scorer if norm_dtype == 'binary' else multiclass_scorer
        else: # continuous
            estimator, scoring = ridge, 'r2'
            
        # Cross validation
        check = checker(solo_embed_names, y, norm_dtype, meta, outer_cv, norm_name)
        if check == 'pass':
            scores = k_fold_cross_val(estimator, X, y, outer_cv, scoring, n_jobs)
        else:
            scores = [np.nan] * outer_cv
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(y))
        for i, score in enumerate(scores):
            rca.append([embed_name, norm_name, train_n, i + 1, score, check])
            
        # Printing
        to_print.append([embed_name, scores.mean(), scores.std(), check])
    to_print = pd.DataFrame(to_print, columns=['embed', 'r2_mean', 'r2_std', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10).reset_index(drop=True))
    print('--------------------------------')
 
 
rca = pd.DataFrame(rca, columns=['embed', 'norm', 'train_n', 'fold', 'r2_mean', 'check'])
rca.to_csv('../../data/final/rca_ensemb.csv', index=False)
rca

  0%|          | 0/292 [00:00<?, ?it/s]

Freq_HAL:
